In [11]:
%cd ../..
# %pip install .
import time
from os import environ
# offset = Uniform(-1, 1) * Range(90, 180) deg

# ego = Car on drivableRoad,
#         facing offset relative to roadDirection,
#         with visibleDistance 50,
#         with viewAngle 135 deg

# otherCar = Car on visible road,
#             facing Range(-15, 15) deg relative to roadDirection

# require (distance from ego to otherCar) < 10

/home/yongming/workspace/research


In [12]:
environ['AP_PORT'] = str(input('port')) # README command uses port=25432

port25432


In [ ]:
"""
with (
  select *
  from item_traj
  where item_traj.object_type = 'car'
) as cars

select *
from cars as otherCar
join Cameras as cam on t1.cameraId = Cameras.id
where
  TODO: ego facing relative to road direction????
  DISTANCE(cam.egoTranslation, otherCar.trajCentroids, cam.timestamp) < 50 AND
  viewAngle(otherCar.traj, cam.egoHeading, cam.ego_Translation, cam.timestamp) < 135 AND
  contained(otherCar.traj, road_segment_type("visibleRoad"), cam.timestamp) and
  facingRelative(otherCar.heading, road_direction(valueAtTimestamp(otherCar.trajCentroids, cam.timestamp)), cam.timestamp) >= -15 AND
  facingRelative(otherCar.heading, road_direction(valueAtTimestamp(otherCar.trajCentroids, cam.timestamp)), cam.timestamp) < 15 AND
  DISTANCE(cam.egoTranslation, otherCar.centroid, cam.timestamp) < 10 AND
"""

In [13]:
### Prepare the world for queries ###
from apperception.world import empty_world
import numpy as np
# directly ingest the mini dataset and boston roadnetwork if needed
# from apperception.database import database
# database.reset()
# from benchmarks.ingest_scenic_data import ingest_data
# ingest_data()

name = 'ScenicWorld' # world name
world = empty_world(name=name)

### Query ###
from apperception.utils import F
# world = world.filter(lambda obj: obj.object_type == 'vehicle.car')
random_sample = np.random.choice([-1,1],1)[0]
facing_range = (random_sample*90, random_sample*180)
def pred(otherCar, cam):
    return (
        # TODO: ego facing `offset` relative to road direction??
        F.distance(cam.ego, obj1, cam.timestamp) < 50 and 
        F.view_angle(obj1, cam.ego, cam.timestamp) < 67.5 and 
        F.facing_relative(cam.ego, F.road_direction(cam.ego, cam.timestamp, cam.ego), cam.timestamp) > {min(facing_range)} and 
        F.facing_relative(cam.ego, F.road_direction(cam.ego, cam.timestamp, cam.ego), cam.timestamp) < {max(facing_range)} and 
        F.contained(cam.ego, F.road_segment('road')) and 
        F.contained(obj1.traj, F.road_segment('road'), cam.timestamp) and 
        F.facing_relative(obj1, F.road_direction(obj1.traj, cam.timestamp, cam.ego), cam.timestamp) >= -15 and 
        F.facing_relative(obj1, F.road_direction(obj1.traj, cam.timestamp, cam.ego), cam.timestamp) < 15 and
        F.distance(cam.ego, obj1, cam.timestamp) < 10
    )
world = world.filter("lambda obj1, cam: " + 
        "F.like(obj1.object_type, 'vehicle%') and " +
        "F.distance(cam.ego, obj1, cam.timestamp) < 50 and " +
        "F.view_angle(obj1, cam.ego, cam.timestamp) < 67.5 and " +
        f"F.facing_relative(cam.ego, F.road_direction(cam.ego, cam.timestamp, cam.ego), cam.timestamp) > -180 and " +
        f"F.facing_relative(cam.ego, F.road_direction(cam.ego, cam.timestamp, cam.ego), cam.timestamp) < -90 and " +
        "F.contained(cam.ego, F.road_segment('road')) and " +
        "F.contained(obj1.traj, F.road_segment('road'), cam.timestamp) and " +
        "F.facing_relative(obj1, F.road_direction(obj1.traj, cam.timestamp, cam.ego), cam.timestamp) >= -15 and " +
        "F.facing_relative(obj1, F.road_direction(obj1.traj, cam.timestamp, cam.ego), cam.timestamp) < 15 and " +
        "F.distance(cam.ego, obj1, cam.timestamp) < 10")

start = time.time()
# keys = world.get_traj_key()
id_time_camId_filename = world.get_id_time_camId_filename(num_joined_tables=1)

end = time.time()
print(format(end-start))

execute: filter
execute: get_id_time_camId_filename
get_id_time_camId_filename 
        SELECT table_0.itemId, cameras.timestamp, cameras.cameraId, cameras.filename
        FROM (SELECT * FROM item_general_trajectory) as table_0
        
        JOIN Cameras USING (cameraId)
        WHERE ((table_0.objectType LIKE 'vehicle%') AND (distance(Cameras.egoTranslation, table_0.trajCentroids, Cameras.timestamp)<50) AND (viewAngle(table_0.trajCentroids, Cameras.egoHeading, Cameras.egoTranslation, Cameras.timestamp)<67.5) AND (facingRelative(Cameras.egoHeading, roadDirection(Cameras.egoTranslation, Cameras.timestamp, Cameras.egoHeading), Cameras.timestamp)>(-180)) AND (facingRelative(Cameras.egoHeading, roadDirection(Cameras.egoTranslation, Cameras.timestamp, Cameras.egoHeading), Cameras.timestamp)<(-90)) AND contained(Cameras.egoTranslation, roadSegment('road')) AND contained(table_0.trajCentroids, roadSegment('road'), Cameras.timestamp) AND (facingRelative(table_0.itemHeadings, roadDirection(

In [14]:
resultImages = dict() # maping from image -> (itemId, timestamp, camId) that it was found at
for result in id_time_camId_filename:
    itemId, timestamp, camId, filename = result
    filename = filename.split("/")[-1] # use split so that prefix path is not included in filename
    resultImages[filename] = (itemId, timestamp, camId)

In [15]:
from matplotlib import pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
import os
import random

experiment_data_dir =  "data/nuscenes/experiment_data"
# experiment_data_dir = '/work/apperception/data/nuScenes/full-dataset-v1.0/Trainval/experiment_data'
def show_images(images, sample=None):
    if sample is not None:
        images = [i for i in images]
        random.shuffle(images)
        images = images[:sample]
    
    plt.figure(figsize=(60,30))
    columns = 3

    for i, image in enumerate(images):
        print("image", image)
        img = mpimg.imread(os.path.join(experiment_data_dir, image))
        print("loaded")
        plt.subplot(len(images) // columns + 1, columns, i + 1)
        plt.imshow(img)

In [16]:
show_images(list(resultImages.keys()))

image n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657491612404.jpg


FileNotFoundError: [Errno 2] No such file or directory: 'data/nuscenes/experiment_data/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657491612404.jpg'

<Figure size 4320x2160 with 0 Axes>

In [ ]:
expected_imgs = ["n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657491612404.jpg", "n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657492112404.jpg"]
show_images(expected_imgs)

In [ ]:
### Getting info about the expected images
from apperception.database import database
for img in expected_imgs:
    prefix = "samples/CAM_FRONT/"
    query = f"""SELECT Distinct \'Cameras: \', Cameras.filename, roadDirection(Cameras.egoTranslation, Cameras.timestamp, Cameras.egoHeading), 
    Cameras.egoHeading, facingRelative(Cameras.egoHeading, roadDirection(Cameras.egoTranslation, Cameras.timestamp, Cameras.egoHeading), Cameras.timestamp),
     ST_X(Cameras.egoTranslation), ST_Y(Cameras.egoTranslation), ST_Z(Cameras.egoTranslation),
    \'Cars: \', table_0.itemId, getX(table_0.trajCentroids, Cameras.timestamp), getY(table_0.trajCentroids, Cameras.timestamp), 
    ST_Z(valueAtTimestamp(table_0.trajCentroids, Cameras.timestamp)), 
    \'Distance: \', distance(Cameras.egoTranslation, table_0.trajCentroids, Cameras.timestamp),
    \'CarHeading:\', valueAtTimestamp(table_0.itemHeadings, Cameras.timestamp),
    \'RoadDirection for car:\', roadDirection(table_0.trajCentroids, Cameras.timestamp, valueAtTimestamp(table_0.itemHeadings, Cameras.timestamp)::real),
    \'facingRelative:\', facingRelative(table_0.itemHeadings, roadDirection(table_0.trajCentroids, Cameras.timestamp, valueAtTimestamp(table_0.itemHeadings, Cameras.timestamp)::real), Cameras.timestamp) 
    FROM Item_General_Trajectory AS table_0, Cameras
                WHERE Cameras.filename = \'{prefix + img}\' AND table_0.cameraId = Cameras.cameraId AND table_0.objectType LIKE 'vehicle%'"""
    database.cursor.execute(query)
    result = database.cursor.fetchall()
    print(result)

In [ ]:
from apperception.database import database
query = f"""SELECT elementId FROM SegmentPolygon WHERE contained(st_point(1269.1763045216746, 911.0066718975124), elementPolygon)
            AND (SELECT id FROM Road WHERE id = elementId) IS NOT NULL;"""
database.cursor.execute(query)
result = database.cursor.fetchall()
print(result)

In [ ]:
from apperception.database import database
query = f"""SELECT ST_Distance(st_point(1269.1763045216746, 911.0066718975124), st_point(1248.3893859608852, 947.7008533706982));"""
database.cursor.execute(query)
result = database.cursor.fetchall()
print(result)

In [ ]:
122.09787+48.640766

In [ ]:
360-(122.09787+48.640766)